## Summary: 

 We estimate the age (DateBP) of the genomes in this notebook.

## Solution Steps: 
1. Accessing the data

2. Exploring the data (initial data analysis)

  2.1. Exploring feature and target distribution
  
3. Preparing the data

  3.1. Cleaning the data

  3.2. Filtering important features

  3.3. Checking missing values

  3.4. Checking for correlation and important features

  3.5. Feature engineering (feature extraction, feature selection)

4. Training model

5. Using the model (Prediction)

6. Evaluating the model (accuracy)

## **Importing libraries**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import zscore,norm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import random
import scipy.stats
import pickle
import warnings
warnings.filterwarnings('ignore')

## **Accessing the data** ##

We have two main datasets, **Ancients** and **Moderns**.

 Ancient genomes from Europe and Asia are found in Ancients, while Euroasian genomes with a DataBP of 10 are found in Moderns.

**Note:** 
Dataset files should be available on data folder (Euroasian.xlsx, Euroasian_modern_samples.csv, 20130606_sample_info.xlsx, Test Cases.xlsxm Reich dataset V50.xlsx) 

In [3]:
baseDir='data'
#ancient samples
dataset =  pd.ExcelFile(f"{baseDir}/Euroasian - Dataset_tims.xlsx")
Euroasian = pd.read_excel(dataset, 'Euroasian')
Ancients = Euroasian[[ 'AncientComponent1', 'AncientComponent2','AncientComponent3', 'AncientComponent4', 'AncientComponent5', 'ModernComponent1',
                'ModernComponent2', 'ModernComponent3', 'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]'
                ,'Method for Determining Date; unless otherwise specified, calibrations use 95.4% intervals from OxCal v4.4.2 Bronk Ramsey (2009); r:5; Atmospheric data from Reimer et al (2020)'
                , 'Sample ID', 'Country'
                ,'Date standard deviation in BP [OxCal sigma for a direct radiocarbon date, and standard deviation of the uniform disribution between the two bounds for a contextual date]'              
]]

Ancients.rename(columns={
    'Method for Determining Date; unless otherwise specified, calibrations use 95.4% intervals from OxCal v4.4.2 Bronk Ramsey (2009); r:5; Atmospheric data from Reimer et al (2020)':'Dating',
    'Date standard deviation in BP [OxCal sigma for a direct radiocarbon date, and standard deviation of the uniform disribution between the two bounds for a contextual date]':'STD',
    'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]':'Mean date (BP)'
},inplace=True)

# modern samples (DateBP = 10)
modern = pd.read_csv(f"{baseDir}/Euroasian_modern_samples.csv")
modern = modern[[ 'AncientComponent1', 'AncientComponent2','AncientComponent3','AncientComponent4', 'AncientComponent5', 'ModernComponent1',
                                'ModernComponent2','ModernComponent3', 'DateBP', 'Sample ID']] 
modern.rename(columns = {'DateBP':'Mean date (BP)'}, inplace=True) 

#modern samples' annotation
modern_annotation = pd.read_excel(f"{baseDir}/20130606_sample_info.xlsx")
modern_annotation.rename(columns = {'Sample':'Sample ID'}, inplace=True) 
modern_annotation.rename(columns = {'Population':'Country'}, inplace=True) 
Moderns = pd.merge(modern_annotation, modern, on=["Sample ID"])
Moderns['Dating'] = np.nan
Moderns = Moderns[['AncientComponent1', 'AncientComponent2','AncientComponent3', 'AncientComponent4', 'AncientComponent5', 'ModernComponent1',
                'ModernComponent2', 'ModernComponent3', 'Mean date (BP)', 'Dating', 'Sample ID', 'Country']]

#test_cases
test_cases = pd.ExcelFile(f"{baseDir}/Test Cases.xlsx")
Brandysek_inds = pd.read_excel(test_cases, 'Brandysek inds')
Brandysek_inds = Brandysek_inds.iloc[0:14,:]
Brandysek_inds.rename(columns={'Version ID':'Sample ID'},inplace=True)

riech = pd.read_excel(f"{baseDir}/Reich dataset V50.xlsx")
riech.rename(columns = {'Version ID':'Sample ID'}, inplace=True) 

Moderns['STD'] = 0 

## **Exploring the data (statistical initial analysis)** ##

In [4]:
print(Ancients.info())
display(Ancients.head(3))
print(Moderns.info())
display(Moderns.head(3))

In [5]:
sns.distplot(Ancients['Mean date (BP)'], fit = norm) # Plot the distribution with a histogram and maximum likelihood gaussian distribution fit.
fig = plt.figure()
stats.probplot(Ancients['Mean date (BP)'], plot=plt) # Generates a probability plot of sample data against the quantiles of a specified theoretical distribution (the normal distribution by default).
"""Skewness is a measure of symmetry, or more precisely, the lack of symmetry.
 Kurtosis is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution.
 That is, data sets with high kurtosis tend to have heavy tails, or outliers."""

## **Cleaning the data**

In [6]:
def outlier_by_zscore(df:pd.DataFrame,column,outliers):
    df['is_outlier'] = zscore(df[column])
    is_outlier = df['is_outlier'].apply(lambda x: x <= -3 or x >= 3)
    df.drop('is_outlier',axis=1,inplace=True)
    return df[~is_outlier],outliers.append(df[is_outlier])

def preprocess(df:pd.DataFrame,outliers):
    df,outliers = outlier_by_zscore(df,'Mean date (BP)',outliers)
    df,outliers = outlier_by_zscore(df,'AncientComponent1',outliers)  
    df,outliers = outlier_by_zscore(df,'AncientComponent2',outliers)
    df,outliers = outlier_by_zscore(df,'AncientComponent3',outliers) 
    df,outliers = outlier_by_zscore(df,'AncientComponent4',outliers)
    df,outliers = outlier_by_zscore(df,'AncientComponent5',outliers)
    df,outliers = outlier_by_zscore(df,'ModernComponent1',outliers)
    df,outliers = outlier_by_zscore(df,'ModernComponent2',outliers)
    df,outliers = outlier_by_zscore(df,'ModernComponent3',outliers)

    outliers11 = df[df.STD >= 400]
    outliers = outliers.append(outliers11)
    df = df[df.STD <400]
    return df,outliers
outliers:pd.DataFrame = pd.DataFrame()
Ancients,outliers = preprocess(Ancients,outliers)


## **Checking the missing values** 

In [7]:
print(Ancients.isnull().sum()) # No missing values are found
print(Moderns1.isnull().sum()) # No missing values are found

In [8]:
sns.distplot(Ancients['Mean date (BP)'], fit = norm)
fig = plt.figure()
stats.probplot(Ancients['Mean date (BP)'], plot = plt)

## **Checking for correlation and important features**

In [9]:
sns.pairplot(Ancients.iloc[:,:])

In [10]:
corr = Ancients.corr().round(2)
sns.heatmap(data = corr, annot=True)
plt.show() #the features are not highly correlated

##**Feature engineering**

In [11]:
#creating new columns based on the values of existing ones
Ancients['mean_Ancient_columns'] = Ancients.loc[:,['AncientComponent1',	'AncientComponent2',	'AncientComponent3',	'AncientComponent4',	'AncientComponent5']].mean(axis=1)  
AncientComponent1_mean = Ancients['AncientComponent1'].mean()
Ancients['AncientComponent1_diff_mean'] = abs(Ancients['AncientComponent1'] - AncientComponent1_mean)  
Ancients['AncientComponent1'] = Ancients['AncientComponent1']+(3*Ancients['AncientComponent1_diff_mean'])

Moderns['mean_Ancient_columns'] = Moderns.loc[:,['AncientComponent1',	'AncientComponent2',	'AncientComponent3',	'AncientComponent4',	'AncientComponent5']].mean(axis=1)     
Moderns['AncientComponent1_diff_mean'] = abs(Moderns['AncientComponent1'] - AncientComponent1_mean)
Moderns['AncientComponent1'] = Moderns['AncientComponent1']+(3*Moderns['AncientComponent1_diff_mean'])

## **Splitting the data**

 ## **Train_set, Unseen_data**

In [12]:
#removing test cases IDs
Brandysek_inds_in_Ancients = Ancients[Ancients["Sample ID"].isin(Brandysek_inds["Sample ID"].values)]
Ancients = Ancients.drop(Brandysek_inds_in_Ancients.index)

# In order to get a better result, we stratified the dataset, so we need to add a few more columns.
# creating temporary columns
Ancients['cut'] = pd.qcut(Ancients['Mean date (BP)'], q = 10, duplicates= 'drop',labels=False, precision = 0) #df was divided into 10 subsets based on DateBP column.
Ancients['Country_cut'] = Ancients['cut'].astype(str) + Ancients['Country'].astype(str)
count_freq = dict(Ancients['Country_cut'].value_counts())
Ancients['count_freq'] = Ancients['Country_cut']
Ancients['count_freq'] = Ancients['count_freq'].map(count_freq)
train1 = Ancients[Ancients.count_freq == 1] #shape =((63,10), 63 of them  were repeated once; we keep them for training.
Ancients  = Ancients[Ancients.count_freq >1]

#stratified ancient genomes based on location and age
train_set, test_set = train_test_split(Ancients, test_size = 0.15, random_state = 0, stratify = Ancients['Country_cut']) 
train_set = train_set.append([train1])

# removing temporary columns
train_set = train_set[train_set.columns.difference(['Country_cut', 'count_freq', 'cut'])]
test_set = test_set[test_set.columns.difference(['Country_cut', 'count_freq', 'cut'])]

#stratified modern samples based on location
Moderns['location'] = Moderns['Country'].astype(str)  #{'CDX': 93,'CEU': 99, 'CHB': 103, 'CHS': 105, 'FIN': 99,'GBR': 91,'IBS': 107,'ITU': 102, 'JPT': 104,'KHV': 99,'PJL': 96,
#'STU': 102,'TSI': 107}
modern_for_modeling, modern_for_test = train_test_split(Moderns, test_size = 0.15, random_state = 42, stratify = Moderns['location']) 
modern_for_test = modern_for_test[modern_for_test.columns.difference([ 'location'])]
modern_for_modeling = modern_for_modeling[modern_for_modeling.columns.difference([ 'location'])]

#unseen data, train_set, test_set (validation)
test_set = test_set.append([modern_for_test, Brandysek_inds_in_Ancients])
test_set.sample(frac=1) # shuffle the DataFrame rows
train_set = train_set.append([modern_for_modeling])
train_set =train_set.sample(frac=1)

#dropping Sample ID and Counrty columns    
whole_set = train_set.append([test_set])
whole_set1 = whole_set.copy()
whole_set = whole_set[whole_set.columns.difference(['Sample ID','Dating', 'Country', 'STD'])]
train_set1 = train_set.copy()
train_set = train_set[train_set.columns.difference(['Sample ID', 'Dating', 'Country','STD'])]
test_set1 = test_set.copy()
test_set = test_set[test_set.columns.difference(['Sample ID', 'Dating', 'Country', 'STD'])]

X_train = train_set[train_set.columns.difference(['Mean date (BP)'])]
y_train = train_set['Mean date (BP)']
X_test = test_set[test_set.columns.difference(['Mean date (BP)'])]
y_test = test_set['Mean date (BP)']
X_whole = whole_set[whole_set.columns.difference(['Mean date (BP)'])]
y_whole = whole_set['Mean date (BP)']

print('Data for Modeling: ' + str(train_set.shape))
print('Unseen Data For Predictions' + str(test_set.shape))

Data for Modeling: (4157, 11)
Unseen Data For Predictions(741, 11)


 ## **Training  model**

In [13]:
def cross_validation(df,target,k=10):
  X=df.drop(target,axis=1).values
  y=df[target].values
  kf = KFold(n_splits=k,shuffle=True,random_state=32)
  R2= []
  model = []
  nth_k=1
  nth_K_X_train={}
  rf_reg = RandomForestRegressor()
  for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    nth_K_X_train[nth_k]=X_train
    rf_reg.fit(X_train, y_train)
    y_val_pred = rf_reg.predict(X_val)
    this_k_R2 = rf_reg.score(X_val,y_val)
    R2.append(this_k_R2)
    model.append(rf_reg)
    nth_k+=1 
  return model,R2,nth_K_X_train 

model,R2,nth_K_X_train = cross_validation(df=train_set,target='Mean date (BP)', k=10)

k=10
random.seed(0)
random_fold = random.randrange(0, 10)
random_model = model[random_fold]
random_X_train = nth_K_X_train[random_fold]

# # save the model
# pickle.dump(random_model, open( 'random_model.pkl', 'wb'))

In [14]:
# load the model from disk
loaded_model = pickle.load(open('/content/drive/MyDrive/ml_models/random_model.pkl', 'rb'))

 ## **Prediction**

In [15]:
test_predictions = X_test.copy()
test_predictions['Mean date (BP)'] = y_test
test_predictions['TPS predicted date (BP)'] = loaded_model.predict(X_test).round()
test_predictions = pd.concat([test_set1[['Sample ID', 'Country', 'Dating']],test_predictions],axis=1)
test_predictions_final = test_predictions
test_predictions_final['Mean date (BP)'] = test_predictions_final['Mean date (BP)'].astype('float')
test_predictions_final["difference"] = abs(test_predictions_final['TPS predicted date (BP)']-test_predictions_final['Mean date (BP)'])

Whole_predictions = X_whole.copy()
Whole_predictions['Mean date (BP)'] = y_whole
Whole_predictions['TPS predicted date (BP)'] = loaded_model.predict(X_whole).round()
Whole_predictions = pd.concat([whole_set1[['Sample ID', 'Country', 'Dating']],Whole_predictions],axis=1)
Whole_predictions_final = Whole_predictions
Whole_predictions_final['Mean date (BP)'] = Whole_predictions_final['Mean date (BP)'].astype('float')
Whole_predictions_final["difference"] = abs(Whole_predictions_final['TPS predicted date (BP)']-Whole_predictions_final['Mean date (BP)'])

X_Brandysek = Brandysek_inds_in_Ancients[Brandysek_inds_in_Ancients.columns.difference(['Sample ID', 'Dating', 'Country', 'Mean date (BP)', 'STD'])]
y_Brandysek = Brandysek_inds_in_Ancients['Mean date (BP)']
Brandysek_inds_in_Ancients1 = Brandysek_inds_in_Ancients.copy()
Brandysek_inds_in_Ancients_predictions = X_Brandysek.copy()
Brandysek_inds_in_Ancients_predictions['Mean date (BP)'] = y_Brandysek
Brandysek_inds_in_Ancients_predictions['TPS predicted date (BP)'] = loaded_model.predict(X_Brandysek).round()
Brandysek_inds_in_Ancients_predictions = pd.concat([Brandysek_inds_in_Ancients1[['Sample ID', 'Country', 'Dating']],Brandysek_inds_in_Ancients_predictions],axis=1)
Brandysek_inds_in_Ancients_predictions["difference"] = abs(Brandysek_inds_in_Ancients_predictions['TPS predicted date (BP)']-Brandysek_inds_in_Ancients_predictions['Mean date (BP)'])
Brandysek_inds_in_Ancients_predictions = Brandysek_inds_in_Ancients_predictions[[ 'Sample ID','AncientComponent1', 'AncientComponent2','AncientComponent3', 'AncientComponent4', 'AncientComponent5', 'ModernComponent1',
                'ModernComponent2', 'ModernComponent3', 'Country', 'Dating', 'Mean date (BP)','TPS predicted date (BP)','difference']]


## **Evaluation**

In [16]:
def evaluate(y, yhat, model, X):
    mae = metrics.mean_absolute_error(y, yhat)
    mse = metrics.mean_squared_error(y, yhat)
    rmse = np.sqrt(metrics.mean_squared_error(y, yhat))
    r2_square = model.score(X, y)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')

test_pred = loaded_model.predict(X_test)
train_pred = loaded_model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
evaluate(y_test, test_pred, loaded_model, X_test)

print('Train set evaluation:\n_____________________________________')
evaluate(y_train, train_pred, loaded_model, X_train)    

Test set evaluation:
_____________________________________
MAE: 462.3314035087719
MSE: 596298.8754708503
RMSE: 772.2039079613947
R2 Square 0.8842938730205804
__________________________________
Train set evaluation:
_____________________________________
MAE: 226.7445898484484
MSE: 164492.75260096224
RMSE: 405.57706123616293
R2 Square 0.968424198595618
__________________________________
